In [ ]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    root = '/content/tree_trunk_segmentation/data/'
    device = 'cuda'
    !wget -O treetrunk.zip https://github.com/lacykaltgr/agriculture-image-processing/archive/refs/heads/laci420.zip
    !unzip treetrunk.zip
    !cp -r agriculture-image-processing-laci420/tree_trunk_segmentation/src .
else:
    import sys
    import os
    root = '/data/'
    device = 'mps'
    parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
    sys.path.append(parent_directory)

In [1]:
from torch.utils.data import DataLoader
from tree_trunk_segmentation.src.dataset import XYDataset, load_dataset

x_train, y_train, x_val, y_val = load_dataset(root='data/',
                                              target_size_x=(4000, 3000),
                                              target_size_y=(4000, 500),
                                              val_split_ratio=0.9)
train_dataset = XYDataset(x_train, y_train)
val_dataset = XYDataset(x_val, y_val)

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [2]:
import torch
from torch import nn
from unet import UNet
from utils import EarlyStopper
from tree_trunk_segmentation.src.utils import binary_to_rgb

config = dict(
    in_channels=3,
    out_channels=1,
    hiddens=[4, 8, 16],
    dropouts=[0, 0.3, 0.3],  # hiddens
    maxpools=2,  # hiddens - 1
    kernel_sizes=3,  # 2*hiddens + 3*hiddens + 2
    paddings=[ ['same']*int(3*2), ['same']*int(3*2) + [(1, 0), (1, 1)]],  # 2*hiddens + 3*hiddens + 2
    strides=[[(1, 1)]*int(3*2), [(1, 1)]*int(3*2) + [(1, 3), (1, 2)]],  # 2*hiddens + 3*hiddens
    criterion=nn.BCELoss,
    output_activation=nn.Softmax,
    activation=nn.ReLU,
    post_process=binary_to_rgb,
    dimensions=2,
    device='cuda'
)
unet = UNet(**config)

In [3]:
unet.summary(input_shape=(1, 3, 4000, 3000))

Layer (type:depth-idx)                   Output Shape              Param #
UNet                                     [1, 1, 4000, 500]         --
├─ModuleDict: 1-13                       --                        (recursive)
│    └─Conv2d: 2-1                       [1, 4, 4000, 3000]        112
├─ReLU: 1-2                              [1, 4, 4000, 3000]        --
├─ModuleDict: 1-13                       --                        (recursive)
│    └─Conv2d: 2-2                       [1, 4, 4000, 3000]        148
├─ReLU: 1-4                              [1, 4, 4000, 3000]        --
├─ModuleDict: 1-13                       --                        (recursive)
│    └─BatchNorm2d: 2-3                  [1, 4, 4000, 3000]        8
│    └─Dropout2d: 2-4                    [1, 4, 4000, 3000]        --
│    └─MaxPool2d: 2-5                    [1, 4, 2000, 1500]        --
│    └─Conv2d: 2-6                       [1, 8, 2000, 1500]        296
├─ReLU: 1-6                              [1, 8, 2000, 15

In [4]:
es = EarlyStopper(patience=10)
unet.train_model(train_loader, val_loader, es, num_epochs=1000, learning_rate=0.0001, device='mps')

RuntimeError: MPS backend out of memory (MPS allocated: 12.40 GB, other allocations: 5.17 GB, max allowed: 18.13 GB). Tried to allocate 732.42 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
predictions, results = unet.predict(val_loader, device=device)

In [ ]:
from matplotlib.pyplot import imshow, show
import numpy as np
imshow(x_train[0])
show()

In [ ]:
from src.utils import conf_matrix
conf_matrix(val_dataset, results)

In [ ]:
unet.load_state_dict(torch.load('/Users/laszlofreund/code/ai/satellite-image-segmentation/unet.h5', map_location=device))

In [ ]:
eval = unet.evaluate(val_loader, device=device)